In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyswarms imbalanced-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.6 MB/s eta 0:00:00


In [ ]:
# Step 1: Install required library
!pip install --upgrade gdown

# Step 2: Download file using gdown (from full URL)
import gdown

url = "https://drive.google.com/uc?id=1t0mqjuEHOu_WZWh6wI_As1KvdZWHixHv"
output = "dataset.csv"
gdown.download(url, output, quiet=False)

# Step 3: Load the CSV into pandas
import pandas as pd

df = pd.read_csv("dataset.csv", low_memory=False)
print("✅ Loaded shape:", df.shape)
df.head()


Downloading...
From (original): https://drive.google.com/uc?id=1t0mqjuEHOu_WZWh6wI_As1KvdZWHixHv
From (redirected): https://drive.google.com/uc?id=1t0mqjuEHOu_WZWh6wI_As1KvdZWHixHv&confirm=t&uuid=22629571-9663-41b1-b24c-d6f0bf8d4648
To: /content/dataset.csv
100%|██████████| 435M/435M [00:05<00:00, 81.4MB/s]


✅ Loaded shape: (9446350, 10)


,protocol,flow_duration,src_bytes,dst_bytes,dst_port,total_fwd_pkts,total_bwd_pkts,flow_bytes_s,flow_pkts_s,attack_type
0,udp,0.001055,132.0,164.0,53,NaN,NaN,NaN,NaN,NaN
1,udp,0.036133,528.0,304.0,1024,NaN,NaN,NaN,NaN,NaN
2,udp,0.001119,146.0,178.0,53,NaN,NaN,NaN,NaN,NaN
3,udp,0.001209,132.0,164.0,53,NaN,NaN,NaN,NaN,NaN
4,udp,0.001169,146.0,178.0,53,NaN,NaN,NaN,NaN,NaN


In [ ]:
!pip install imbalanced-learn

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


Feature Alignment

In [ ]:
# Step 1: Install gdown if needed
!pip install --upgrade gdown

# Step 2: Define file IDs and download them
import gdown

file_ids = {
    "CIC-IDS2017": "1va0WNVo05ue2v9AIwCe3NWBfZ_j2TzPX",
    "CIC-IDS2018": "1URSMeZjRmw_7h4gpVqHdroMDv-kuDIUe",
    "CIC-IDS2019": "1vhECd8KzkT6pA4gis82lvx71pt6K5Kfj",  # updated ID
    "UNSW-NB15":   "1n45tmx5_GTTaqgLr8kv05HEOELXni62e",
    "TON_IoT":     "14smq_StPP4WEDkPMfJO374fl1_wM-6-Z"
}

# Step 3: Download and save with readable names
for name, file_id in file_ids.items():
    gdown.download(f"https://drive.google.com/uc?id={file_id}", f"{name}.csv", quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1va0WNVo05ue2v9AIwCe3NWBfZ_j2TzPX
From (redirected): https://drive.google.com/uc?id=1va0WNVo05ue2v9AIwCe3NWBfZ_j2TzPX&confirm=t&uuid=5430dc4e-c30e-4a5b-b510-1c1a52e9b864
To: /content/CIC-IDS2017.csv
100%|██████████| 156M/156M [00:01<00:00, 90.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1URSMeZjRmw_7h4gpVqHdroMDv-kuDIUe
To: /content/CIC-IDS2018.csv
100%|██████████| 30.9M/30.9M [00:00<00:00, 125MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1vhECd8KzkT6pA4gis82lvx71pt6K5Kfj
From (redirected): https://drive.google.com/uc?id=1vhECd8KzkT6pA4gis82lvx71pt6K5Kfj&confirm=t&uuid=1c073b9b-e4eb-464f-8218-009e80cbd3e1
To: /content/CIC-IDS2019.csv
100%|██████████| 156M/156M [00:01<00:00, 99.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1n45tmx5_GTTaqgLr8kv05HEOELXni62e
From (redirected): https://drive.google.com/uc?id=1n45tmx5_GTTaqgLr8kv05HEOELXni62e&confirm=t&uuid=de83008d-a

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# Colab working directory
base = Path("/content")

sources = {
    "UNSW":    base / "unsw_nb15.csv",
    "TON-IoT": base / "ton_iot.csv",
    "CIC17":   base / "CIC-IDS2017.csv",
    "CIC18":   base / "CIC-IDS2018.csv",
    "CIC19":   base / "combined_cic19.csv",
}

final_cols = [
    "protocol", "flow_duration", "src_bytes", "dst_bytes", "dst_port",
    "total_fwd_pkts", "total_bwd_pkts", "flow_bytes_s", "flow_pkts_s",
    "attack_type"
]

def load_and_standardize(name, path):
    df = pd.read_csv(path, encoding='ISO-8859-1', low_memory=False, on_bad_lines='skip')
    df.columns = df.columns.str.strip()

    # Rename per source
    if name == "UNSW":
        df = df.rename(columns={
            'proto': 'protocol', 'dur': 'flow_duration',
            'sbytes':'src_bytes','dbytes':'dst_bytes',
            'dsport':'dst_port','attack_cat':'attack_type'
        })
    elif name == "TON-IoT":
        df = df.rename(columns={
            'proto':'protocol','duration':'flow_duration',
            'src_bytes':'src_bytes','dst_bytes':'dst_bytes',
            'dst_port':'dst_port','attack':'attack_type'
        })
    elif name == "CIC17":
        df = df.rename(columns={
            'Flow Duration':'flow_duration','Total Fwd Packets':'total_fwd_pkts',
            'Total Backward Packets':'total_bwd_pkts','Flow Bytes/s':'flow_bytes_s',
            'Flow Packets/s':'flow_pkts_s','Destination Port':'dst_port',
            'TYPE':'attack_type'
        })
    elif name == "CIC18":
        df = df.rename(columns={
            'Flow Duration':'flow_duration','TotLen Fwd Pkts':'total_fwd_pkts',
            'TotLen Bwd Pkts':'total_bwd_pkts','Flow Byts/s':'flow_bytes_s',
            'Flow Pkts/s':'flow_pkts_s','Dst Port':'dst_port','Label':'attack_type'
        })
    elif name == "CIC19":
        df = df.rename(columns={
            'Flow Duration':'flow_duration','Total Fwd Packets':'total_fwd_pkts',
            'Total Backward Packets':'total_bwd_pkts','Flow Bytes/s':'flow_bytes_s',
            'Flow Packets/s':'flow_pkts_s','Label':'attack_type'
        })
    else:
        raise ValueError(f"Unknown source: {name}")

    # Now select only columns that exist
    existing = [c for c in final_cols if c in df.columns]
    df = df[existing].copy()

    # Add missing cols as NaN
    for col in final_cols:
        if col not in df.columns:
            df[col] = np.nan

    # Reorder
    return df[final_cols]

# Load & concatenate
frames = []
for name, path in sources.items():
    if path.exists():
        df_std = load_and_standardize(name, path)
        print(f"{name}: selected columns {df_std.columns.tolist()}")
        frames.append(df_std)
    else:
        print(f"⚠️  {name} not found, skipping.")

combined = pd.concat(frames, ignore_index=True)
print("✅ Combined shape:", combined.shape)

# Save
combined.to_csv(base / "final_combined_dataset.csv", index=False)
print("✅ Saved to final_combined_dataset.csv")


⚠️  UNSW not found, skipping.
⚠️  TON-IoT not found, skipping.
CIC17: selected columns ['protocol', 'flow_duration', 'src_bytes', 'dst_bytes', 'dst_port', 'total_fwd_pkts', 'total_bwd_pkts', 'flow_bytes_s', 'flow_pkts_s', 'attack_type']
CIC18: selected columns ['protocol', 'flow_duration', 'src_bytes', 'dst_bytes', 'dst_port', 'total_fwd_pkts', 'total_bwd_pkts', 'flow_bytes_s', 'flow_pkts_s', 'attack_type']
⚠️  CIC19 not found, skipping.
✅ Combined shape: (1479946, 10)
✅ Saved to final_combined_dataset.csv


In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Paths where you downloaded the CSVs (update if needed)
paths = {
    "CIC-IDS2017": "/content/CIC-IDS2017.csv",
    "CIC-IDS2018": "/content/CIC-IDS2018.csv",
    "CIC-IDS2019": "/content/CIC-IDS2019.csv",
    "UNSW-NB15":   "/content/UNSW-NB15.csv",
    "TON_IoT":     "/content/TON_IoT.csv"
}

# Collect the set of columns for each dataset
feature_counts = {}
for name, path in paths.items():
    df = pd.read_csv(path, low_memory=False, on_bad_lines='skip', encoding='ISO-8859-1')
    df.columns = df.columns.str.strip().str.lower()
    feature_counts[name] = len(df.columns)

# Build a summary DataFrame
df_feat = pd.DataFrame.from_dict(feature_counts, orient='index', columns=['Original Feature Count'])
df_feat['Common Retained'] = df_feat['Original Feature Count'].apply(lambda x: len(common_features))
df_feat = df_feat.sort_index()
print(df_feat)


NameError: name 'common_features' is not defined

In [ ]:
plt.figure(figsize=(8, 4))
plt.bar(df_feat.index, df_feat['Original Feature Count'], label='Original', alpha=0.6)
plt.bar(df_feat.index, df_feat['Common Retained'],   label='Retained',  alpha=0.8)
plt.ylabel("Number of Features")
plt.title("Feature Count Before vs. After Alignment")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load the raw combined CSV (before any feature alignment)
df_before = pd.read_csv("/content/dataset.csv", low_memory=False)

# 2. Clean column names
df_before.columns = df_before.columns.str.strip().str.lower()

# 3. Drop rows missing the raw label
df_before = df_before.dropna(subset=['attack_type'])

# 4. (Optional) Normalize label strings
df_before['attack_type'] = (
    df_before['attack_type']
      .astype(str)
      .str.strip()
      .str.upper()
)

# 5. Plot the class distribution
plt.figure(figsize=(10, 6))
sns.countplot(
    y='attack_type',
    data=df_before,
    order=df_before['attack_type'].value_counts().index
)
plt.title("Class Distribution Before Any Preprocessing")
plt.xlabel("Sample Count")
plt.ylabel("Attack Type")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load the raw combined file
raw_path = "/content/dataset.csv"
df_raw = pd.read_csv(raw_path, low_memory=False)

# 2. Clean column names
df_raw.columns = df_raw.columns.str.strip().str.lower()

# 3. Peek at attack_type values
print("Sample attack_type values:", df_raw['attack_type'].unique()[:10])
print("Missing attack_type count:", df_raw['attack_type'].isna().sum())
print("Non-missing attack_type count:", df_raw['attack_type'].notna().sum())

# 4. Filter out true NaNs (and the string "nan" if present)
df_plot = df_raw[
    df_raw['attack_type'].notna() &
    (df_raw['attack_type'].astype(str).str.lower() != 'nan')
].copy()
print("Rows with valid attack_type:", len(df_plot))

# 5. Normalize labels (trim whitespace, uppercase)
df_plot['attack_type'] = df_plot['attack_type'].astype(str).str.strip().str.upper()

# 6. Plot the top 20 attack types
top_n = 20
order = df_plot['attack_type'].value_counts().nlargest(top_n).index

plt.figure(figsize=(10, 6))
sns.countplot(
    y='attack_type',
    data=df_plot,
    order=order,
    palette='viridis'
)
plt.title(f"Top {top_n} Attack Types Before Preprocessing")
plt.xlabel("Sample Count")
plt.ylabel("Attack Type")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Load the aligned combined dataset
df = pd.read_csv("/content/dataset.csv", low_memory=False)

# 2. Missing‐value summary (before)
num_rows = len(df)
missing = df.isna().sum()
missing_pct = (missing / num_rows * 100).round(2)
missing_summary = pd.DataFrame({
    "Missing Count": missing,
    "Missing %":     missing_pct
})
print(missing_summary)

# 3. Plot missing % before
plt.figure(figsize=(8,4))
missing_summary["Missing %"].sort_values(ascending=False).plot(kind="bar", color="skyblue")
plt.title("Missing % per Column (Before)")
plt.ylabel("% Missing")
plt.tight_layout()
plt.show()

# 4. Drop rows with >30% missing values (more than 3 of 10)
df_clean = df[df.isna().sum(axis=1) <= 3].copy()
print(f"Rows before: {num_rows:,} → after drop: {len(df_clean):,}")

# 5. Impute numeric cols with median
num_cols = df_clean.select_dtypes(include=[np.number]).columns
for col in num_cols:
    df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# 6. Impute categorical cols with mode or placeholder
for col in ["protocol", "attack_type"]:
    if col in df_clean.columns:
        if df_clean[col].notna().any():
            mode_val = df_clean[col].mode().iloc[0]
        else:
            mode_val = "UNKNOWN"
        df_clean[col] = df_clean[col].fillna(mode_val)

# 7. Missing‐value summary (after)
missing_after = df_clean.isna().sum()
missing_after_pct = (missing_after / len(df_clean) * 100).round(2)
missing_summary2 = pd.DataFrame({
    "Missing Count": missing_after,
    "Missing %":     missing_after_pct
})
print(missing_summary2)

# 8. Plot missing % after
plt.figure(figsize=(8,4))
missing_summary2["Missing %"].sort_values(ascending=False).plot(kind="bar", color="coral")
plt.title("Missing % per Column (After Imputation)")
plt.ylabel("% Missing")
plt.tight_layout()
plt.show()

# 9. Save cleaned data
df_clean.to_csv("/content/cleaned_combined_dataset.csv", index=False)
print("✅ Cleaned data saved.")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

# 1. Load the cleaned dataset
df = pd.read_csv("/content/dataset.csv", low_memory=False)

# 2. Drop any rows with missing or empty attack_type
df = df[df['attack_type'].notna() & (df['attack_type'].astype(str).str.strip() != '')].copy()

# 3. Normalize label strings
df['attack_type'] = (
    df['attack_type']
      .astype(str)
      .str.strip()
      .str.upper()
)

# 4. Consolidate into 10 classes
label_map = {
    # Normal / Benign
    'BENIGN':      'NORMAL',
    'NORMAL':      'NORMAL',
    '0.0':         'NORMAL',
    '1.0':         'NORMAL',
    # DoS variants
    'DOS ATTACKS-HULK': 'DOS',
    'DOS HULK':         'DOS',
    'DOS':              'DOS',
    'DDOS':             'DOS',
    # Exploits
    'EXPLOITS':   'EXPLOITS',
    # Fuzzers
    'FUZZERS':    'FUZZERS',
    # Generic
    'GENERIC':    'GENERIC',
    # Reconnaissance
    'RECONNAISSANCE': 'RECONNAISSANCE',
    # Backdoor
    'BACKDOOR':   'BACKDOOR',
    # Shellcode
    'SHELLCODE':  'SHELLCODE',
    # Worms
    'WORMS':      'WORMS'
}
df['attack_type'] = df['attack_type'].map(label_map).fillna('UNKNOWN')

# 5. Encode the consolidated labels
le = LabelEncoder()
df['label_enc'] = le.fit_transform(df['attack_type'])
mapping = {cls: idx for idx, cls in enumerate(le.classes_)}
print("Encoded Label Mapping:\n", mapping)

# 6. Plot the consolidated class distribution
plt.figure(figsize=(10, 6))
sns.countplot(
    y='attack_type',
    data=df,
    order=df['attack_type'].value_counts().index,
    palette='tab10'
)
plt.title("Class Distribution After Label Consolidation")
plt.xlabel("Sample Count")
plt.ylabel("Attack Class")
plt.tight_layout()
plt.show()

# 7. Save for next steps
df.to_csv("/content/label_cleaned_dataset.csv", index=False)


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt

# 1. Load the cleaned dataset
df = pd.read_csv("/content/dataset.csv", low_memory=False)

# 2. Drop columns with 100% missing (src_bytes, dst_bytes)
for col in ["src_bytes", "dst_bytes"]:
    if df[col].isna().all():
        df.drop(columns=[col], inplace=True)
        print(f"Dropped {col} (all missing)")

# 3. Encode 'protocol' categorical feature
le_proto = LabelEncoder()
df['protocol_enc'] = le_proto.fit_transform(df['protocol'].astype(str))

# 4. (Re)encode 'attack_type' if not already numeric
if 'label_enc' not in df.columns:
    le_lbl = LabelEncoder()
    df['label_enc'] = le_lbl.fit_transform(df['attack_type'].astype(str))

# 5. Scale numeric features
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
# Exclude encoded labels from scaling
numeric_cols = [c for c in numeric_cols if c not in ['protocol_enc', 'label_enc']]

scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
print("Scaled numeric features:", numeric_cols)

# 6. Save final preprocessed dataset
final_cols = numeric_cols + ['protocol_enc', 'label_enc']
df[final_cols].to_csv("/content/final_preprocessed_dataset.csv", index=False)
print("✅ Final preprocessed dataset saved.")

# 7. (Optional) Plot one feature before vs. after scaling for demonstration
#    Here we re-load unscaled for comparison
df_raw = pd.read_csv("/content/cleaned_combined_dataset.csv", low_memory=False)
feat = numeric_cols[0]  # e.g., 'flow_duration'

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.hist(df_raw[feat].dropna(), bins=50)
plt.title(f"{feat} Before Scaling")
plt.subplot(1,2,2)
plt.hist(df[feat], bins=50)
plt.title(f"{feat} After Scaling")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

df = pd.read_csv("/content/label_cleaned_dataset.csv", low_memory=False)
print("Columns in label-cleaned file:", df.columns.tolist())


In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Reload after inspection
df = pd.read_csv("/content/label_cleaned_dataset.csv", low_memory=False)

# If raw 'protocol' exists, encode it; otherwise assume 'protocol_enc' is present
if 'protocol' in df.columns:
    le_proto = LabelEncoder()
    df['protocol_enc'] = le_proto.fit_transform(df['protocol'].astype(str))
else:
    print("✅ No raw 'protocol' column—using existing 'protocol_enc'")

# Identify numeric columns to scale (exclude any encodings)
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
exclude = [c for c in ['protocol_enc', 'label_enc'] if c in numeric_cols]
scale_cols = [c for c in numeric_cols if c not in exclude]

# Scale
scaler = StandardScaler()
df[scale_cols] = scaler.fit_transform(df[scale_cols])
print("Scaled features:", scale_cols)

# Save final preprocessed dataset
final_cols = scale_cols + ['protocol_enc', 'label_enc']
df[final_cols].to_csv("/content/final_preprocessed_dataset.csv", index=False)
print("✅ Saved final preprocessed dataset with columns:", final_cols)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the final preprocessed dataset
df_final = pd.read_csv("/content/final_preprocessed_dataset.csv", low_memory=False)

# Identify feature columns (excluding label_enc)
feature_cols = [col for col in df_final.columns if col != 'label_enc']

# Determine grid size for plots
n = len(feature_cols)
cols = 3
rows = (n + cols - 1) // cols

# Plot histograms for each feature
plt.figure(figsize=(cols * 4, rows * 3))
for i, col in enumerate(feature_cols, 1):
    plt.subplot(rows, cols, i)
    plt.hist(df_final[col].dropna(), bins=50)
    plt.title(col)
    plt.tight_layout()

plt.suptitle("Feature Distributions After Scaling and Encoding", y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from pyswarms.discrete.binary import BinaryPSO
import matplotlib.pyplot as plt

# 1. Load the preprocessed dataset
df = pd.read_csv("/content/final_preprocessed_dataset.csv", low_memory=False)
X = df.drop(columns=['label_enc']).values
y = df['label_enc'].values
feature_names = df.drop(columns=['label_enc']).columns.tolist()

# 2. Define the PSO fitness function
def fitness_function(particles):
    """
    Each particle is a binary array of length n_features.
    We select features where bit == 1, train a classifier,
    and return 1 - mean accuracy (to minimize).
    """
    n_particles = particles.shape[0]
    scores = np.zeros(n_particles)
    for i, particle in enumerate(particles):
        if particle.sum() == 0:
            # if no feature selected, penalize heavily
            scores[i] = 1.0
        else:
            selected_idx = np.where(particle == 1)[0]
            X_sel = X[:, selected_idx]
            # 3‐fold CV for speed; increase for more robust selection
            clf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
            score = cross_val_score(clf, X_sel, y, cv=3, scoring='accuracy').mean()
            scores[i] = 1.0 - score  # PSO minimizes the function
    return scores

# 3. Initialize PSO
from pyswarms.discrete.binary import BinaryPSO

# 3. Initialize PSO with required options
n_features = X.shape[1]
options = {
    'c1': 1.5,    # cognitive coefficient
    'c2': 1.5,    # social coefficient
    'w':  0.7,    # inertia weight
    'k':  30,     # number of neighbors for velocity update
    'p':  2       # Minkowski p-norm (Euclidean distance)
}

pso = BinaryPSO(n_particles=30, dimensions=n_features, options=options)

# 4. Run optimization
best_cost, best_pos = pso.optimize(fitness_function, iters=40)

# 5. Extract selected features
selected_features = [name for bit, name in zip(best_pos, feature_names) if bit == 1]
print("Selected Features (%d of %d):" % (len(selected_features), n_features))
print(selected_features)

# 6. Plot feature‐selection mask
plt.figure(figsize=(10,2))
plt.bar(range(n_features), best_pos, color='teal')
plt.yticks([0,1], ['Excluded','Included'])
plt.xticks(range(n_features), feature_names, rotation=45, ha='right')
plt.title("PSO Selected Feature Mask")
plt.tight_layout()
plt.show()

# 7. Save the reduced dataset
df_reduced = df[selected_features + ['label_enc']]
df_reduced.to_csv("/content/reduced_dataset_pso.csv", index=False)
print("✅ Reduced dataset saved to 'reduced_dataset_pso.csv'")


In [ ]:
# 1. Fill attack_type NaNs with mode
df['attack_type'] = df['attack_type'].fillna(df['attack_type'].mode()[0])

# 2. Encode attack_type → label_enc BEFORE class filtering
from sklearn.preprocessing import LabelEncoder
le_target = LabelEncoder()
df['label_enc'] = le_target.fit_transform(df['attack_type'])

# 3. Now safe to remove rare classes
class_counts = df['label_enc'].value_counts()
valid_classes = class_counts[class_counts >= 2].index
df = df[df['label_enc'].isin(valid_classes)]

# 4. Define X and y
X = df.drop(columns=['attack_type', 'label_enc'])
y = df['label_enc']


In [ ]:
print("Label distribution after cleaning:")
print(y.value_counts())


In [ ]:
min_samples = 100
class_counts = df['label_enc'].value_counts()
valid_classes = class_counts[class_counts >= min_samples].index
df = df[df['label_enc'].isin(valid_classes)]

# Redefine X and y
X = df.drop(columns=['attack_type', 'label_enc'])
y = df['label_enc']


In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x=class_counts.index, y=class_counts.values)
plt.xticks(rotation=90)
plt.title("Class Distribution Before Removing Rare Classes")
plt.ylabel("Sample Count")
plt.xlabel("Label")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.countplot(y='attack_type', data=df, order=df['attack_type'].value_counts().index[:20])
plt.title("Class Distribution Before Preprocessing")
plt.xlabel("Count")
plt.ylabel("Attack Type")
plt.tight_layout()
plt.show()


In [ ]:
import missingno as msno
msno.heatmap(df)
plt.title("Missing Data Heatmap (Before Preprocessing)")
plt.show()


In [ ]:
df_corr = df.select_dtypes(include=['float64', 'int64']).corr()
plt.figure(figsize=(10, 8))
sns.heatmap(df_corr, annot=False, cmap='coolwarm')
plt.title("Feature Correlation Heatmap Before Cleaning")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

print(f"Train: {len(X_train)} samples")
print(f"Val:   {len(X_val)} samples")
print(f"Test:  {len(X_test)} samples")


In [ ]:
# When you first load the CSV
df_full = pd.read_csv("/content/dataset.csv", low_memory=False)

# Optional: preview
df_full.head()


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_11['dst_port'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_11, x='dst_bytes', y='dst_port', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_10['flow_duration'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_10, x='dst_bytes', y='flow_duration', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_9['dst_port'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_9, x='src_bytes', y='dst_port', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['dst_port'].value_counts()
    for x_label, grp in _df_7.groupby('flow_duration')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('flow_duration')
_ = plt.ylabel('dst_port')

In [ ]:
from matplotlib import pyplot as plt
_df_6['dst_bytes'].plot(kind='line', figsize=(8, 4), title='dst_bytes')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
_df_5['src_bytes'].plot(kind='line', figsize=(8, 4), title='src_bytes')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
_df_4.plot(kind='scatter', x='src_bytes', y='dst_bytes', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('dst_port').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('flow_duration').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
_df_1['dst_bytes'].plot(kind='hist', bins=20, title='dst_bytes')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
from matplotlib import pyplot as plt
_df_0['src_bytes'].plot(kind='hist', bins=20, title='src_bytes')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# When you first load the CSV
df_full = pd.read_csv("/content/dataset.csv", low_memory=False)

# Optional: preview
df_full.head()

# Fill missing attack labels with 'Unknown' for before-plot
df_full['attack_type'] = df_full['attack_type'].fillna('Unknown')
before_counts = df_full['attack_type'].value_counts()

# Reuse cleaned DataFrame
df_clean = df.copy()
df_clean['attack_type'] = le_target.inverse_transform(df_clean['label_enc'])
after_counts = df_clean['attack_type'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

fig, axes = plt.subplots(1, 2, figsize=(24, 18), sharey=True)

# Before Cleaning
sns.barplot(y=before_counts.index, x=before_counts.values, ax=axes[0])
axes[0].set_title("Before Cleaning", fontsize=16)
axes[0].set_xlabel("Count")
axes[0].set_ylabel("Attack Type")
axes[0].tick_params(axis='y', labelsize=9)

# After Cleaning
sns.barplot(y=after_counts.index, x=after_counts.values, ax=axes[1])
axes[1].set_title("After Cleaning (<100 Removed)", fontsize=16)
axes[1].set_xlabel("Count")
axes[1].set_ylabel("")
axes[1].tick_params(axis='y', labelsize=9)

plt.suptitle("Attack Type Distribution — Before vs After Cleaning", fontsize=20)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig("attack_type_distribution_comparison.png", dpi=300)
plt.show()


In [ ]:
plt.figure(figsize=(10,8))
# Convert 'protocol' to numerical using Label Encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_clean['protocol_enc'] = le.fit_transform(df_clean['protocol'])

# Select only numerical features for correlation
numerical_features = df_clean.select_dtypes(include=np.number).columns
df_clean_numeric = df_clean[numerical_features]

# Drop original 'protocol' and use the encoded one for correlation
sns.heatmap(df_clean_numeric.drop(columns=['label_enc', 'protocol_enc']).corr(), annot=False, cmap='coolwarm')
plt.title("Feature Correlation Heatmap")
plt.show()

Class Imbalance Ratio Plot


In [ ]:
attack_type_map = {
    "DoS Hulk": "DoS",
    "DoS attacks-Hulk": "DoS",
    "DoS attacks-GoldenEye": "DoS",
    "DoS GoldenEye": "DoS",
    "DoS Slowloris": "DoS",
    "DoS attacks-Slowloris": "DoS",
    "DoS attacks-SlowHTTPTest": "DoS",
    "DoS SlowHTTPTest": "DoS",

    "Web Attack − Brute Force": "Brute Force",
    "Brute Force": "Brute Force",
    "Web Attack − XSS": "XSS",
    "XSS": "XSS",

    "Web Attack − SQL Injection": "SQL Injection",
    "SQL Injection": "SQL Injection",

    "DDoS": "DDoS",
    "DDoS LOIC": "DDoS",
    "DDoS HOIC": "DDoS",

    # Extend as needed...
}


In [ ]:
df_full['attack_type'] = df_full['attack_type'].replace(attack_type_map)


In [ ]:
# Normalize all attack_type labels
df_full['attack_type'] = df_full['attack_type'].str.strip().str.title()
df_clean['attack_type'] = df_clean['attack_type'].str.strip().str.title()


In [ ]:
# Normalize and then apply your label mapping
df_full['attack_type'] = df_full['attack_type'].str.strip().str.title()
df_clean['attack_type'] = df_clean['attack_type'].str.strip().str.title()

attack_type_map = {
    "Dos Hulk": "Dos",
    "Dos Attacks-Hulk": "Dos",
    "Dos Attacks-Goldeneye": "Dos",
    "Dos Goldeneye": "Dos",
    "Dos Slowloris": "Dos",
    "Dos Attacks-Slowloris": "Dos",
    "Dos Attacks-Slowhttptest": "Dos",
    "Dos Slowhttptest": "Dos",

    "Web Attack − Brute Force": "Brute Force",
    "Web Attack − Xss": "Xss",
    "Web Attack − Sql Injection": "Sql Injection",
    # Add more if needed
}

df_full['attack_type'] = df_full['attack_type'].replace(attack_type_map)
df_clean['attack_type'] = df_clean['attack_type'].replace(attack_type_map)


In [ ]:
class_ratios = df_clean['attack_type'].value_counts(normalize=True)

plt.figure(figsize=(14, 6))
sns.barplot(x=class_ratios.index, y=class_ratios.values)
plt.xticks(rotation=90)
plt.title("Class Imbalance Ratios After Cleaning (Merged & Normalized Labels)")
plt.xlabel("attack_type")
plt.ylabel("Ratio")
plt.tight_layout()
plt.savefig("class_ratios_final_cleaned.png", dpi=300)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 🧹 Normalize attack_type in both DataFrames
df_full['attack_type'] = df_full['attack_type'].str.strip().str.title()
df_clean['attack_type'] = df_clean['attack_type'].str.strip().str.title()

# 🔁 Recompute counts after cleaning
before_counts = df_full['attack_type'].value_counts()
after_counts = df_clean['attack_type'].value_counts()

# 📊 Plot
sns.set(style="whitegrid")
fig, axes = plt.subplots(1, 2, figsize=(24, 18), sharey=True)

# BEFORE Cleaning
sns.barplot(y=before_counts.index, x=before_counts.values, ax=axes[0])
axes[0].set_title("Before Cleaning", fontsize=16)
axes[0].set_xlabel("Count")
axes[0].set_ylabel("Attack Type")
axes[0].tick_params(axis='y', labelsize=9)

# AFTER Cleaning
sns.barplot(y=after_counts.index, x=after_counts.values, ax=axes[1])
axes[1].set_title("After Cleaning (<100 Removed)", fontsize=16)
axes[1].set_xlabel("Count")
axes[1].set_ylabel("")
axes[1].tick_params(axis='y', labelsize=9)

# Overall layout
plt.suptitle("Attack Type Distribution — Before vs After Cleaning", fontsize=20)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig("attack_type_distribution_comparison_cleaned.png", dpi=300)
plt.show()


In [ ]:
# Re-import necessary packages after reset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE

# Define simulated feature alignment and consolidation pipeline
common_features = [
    'flow_duration', 'total_fwd_pkts', 'total_bwd_pkts',
    'flow_bytes_s', 'flow_pkts_s', 'fwd_pkt_len_mean', 'bwd_pkt_len_mean'
]

# Simulate loading 5 datasets (replace these paths with actual file paths as needed)
paths = ['unsw_nb15.csv', 'ton_iot.csv', 'cic17.csv', 'cic18.csv', 'cic19.csv']
dfs = []

# Simulate placeholder DataFrame with appropriate structure if not real files exist
for path in paths:
    df_simulated = pd.DataFrame({
        'flow_duration': np.random.rand(100),
        'total_fwd_pkts': np.random.rand(100),
        'total_bwd_pkts': np.random.rand(100),
        'flow_bytes_s': np.random.rand(100),
        'flow_pkts_s': np.random.rand(100),
        'fwd_pkt_len_mean': np.random.rand(100),
        'bwd_pkt_len_mean': np.random.rand(100),
        'attack_cat': np.random.choice(['BENIGN', 'DoS', 'Backdoor', 'Fuzzers'], 100),
        'label': np.random.choice([0, 1], 100)
    })
    dfs.append(df_simulated)

# Step 2: Concatenate datasets
combined = pd.concat(dfs, ignore_index=True)

# Step 3: Missing value handling (simulate NaNs)
combined.loc[combined.sample(frac=0.1).index, 'flow_duration'] = np.nan
threshold = int(0.3 * len(common_features))
combined = combined[combined[common_features].isnull().sum(axis=1) <= threshold]
for col in combined.columns:
    if combined[col].dtype == 'object':
        combined[col].fillna(combined[col].mode()[0], inplace=True)
    else:
        combined[col].fillna(combined[col].median(), inplace=True)

# Step 4: Label consolidation
attack_map = {
    'ANALYSIS': 'Analysis', 'BACKDOOR': 'Backdoor', 'DOS': 'DoS', 'DDOS': 'DoS',
    'EXPLOITS': 'Exploits', 'FUZZERS': 'Fuzzers', 'GENERIC': 'Generic',
    'BENIGN': 'Normal', 'NORMAL': 'Normal', 'RECONNAISSANCE': 'Reconnaissance',
    'SHELLCODE': 'Shellcode', 'WORMS': 'Worms'
}
combined['attack_cat'] = combined['attack_cat'].str.upper().replace(attack_map)
le = LabelEncoder()
combined['attack_cat_enc'] = le.fit_transform(combined['attack_cat'])

# Step 6: Scaling
scaler = StandardScaler()
X = combined[common_features]
X_scaled = scaler.fit_transform(X)

# Step 7: SMOTE
y = combined['attack_cat_enc']
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_scaled, y)

# Plot class distribution before and after SMOTE
plt.figure(figsize=(10, 4))
sns.countplot(x=y, order=np.unique(y))
plt.title("Class Distribution Before SMOTE")
plt.xlabel("Encoded Class")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
sns.countplot(x=y_resampled, order=np.unique(y_resampled))
plt.title("Class Distribution After SMOTE")
plt.xlabel("Encoded Class")
plt.ylabel("Count")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.preprocessing import LabelEncoder

X = X.copy()
if 'protocol' in X.columns:
    le_proto = LabelEncoder()
    X['protocol'] = le_proto.fit_transform(X['protocol'])


In [ ]:
# prompt: save encoded file as new csv

# Assuming 'df_clean' is your cleaned DataFrame with 'label_enc' column
df_clean.to_csv('encoded_dataset.csv', index=False)


In [ ]:
print("📋 Columns in encoded CSV:")
print(df_after.columns.tolist())


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load data
df_before = pd.read_csv("/content/dataset.csv")
df_after = pd.read_csv("/content/encoded_dataset.csv")

# Standardize column names
df_before.columns = df_before.columns.str.strip().str.lower()
df_after.columns = df_after.columns.str.strip().str.lower()

# Clean 'attack_type' column
df_before['attack_type'] = df_before['attack_type'].astype(str).str.strip().str.upper()
df_after['attack_type'] = df_after['attack_type'].astype(str).str.strip().str.upper()

# Group counts
before_counts = df_before['attack_type'].value_counts()
after_counts = df_after.groupby('attack_type')['label_enc'].count()

# Align on union of all class labels
all_labels = sorted(set(before_counts.index).union(after_counts.index))
before = before_counts.reindex(all_labels, fill_value=0)
after = after_counts.reindex(all_labels, fill_value=0)

# Combine into DataFrame
df_plot = pd.DataFrame({
    'Before Encoding': before,
    'After Encoding': after
})

# Plot side-by-side horizontal bars
df_plot.sort_values('Before Encoding', ascending=True).plot(
    kind='barh', figsize=(12, 12),
    color=['steelblue', 'coral'],
    width=0.8
)
plt.title("📊 Class Distribution Before vs After Label Encoding (Clean View)")
plt.xlabel("Sample Count")
plt.ylabel("Attack Type")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.preprocessing import LabelEncoder

if 'protocol' in X_train.columns:
    le_proto = LabelEncoder()
    X_train['protocol'] = le_proto.fit_transform(X_train['protocol'])
    X_val['protocol'] = le_proto.transform(X_val['protocol'])
    X_test['protocol'] = le_proto.transform(X_test['protocol'])


In [ ]:
non_numeric_cols = X_train.select_dtypes(include='object').columns.tolist()
print("Non-numeric columns:", non_numeric_cols)


In [ ]:
for col in ['flow_duration', 'dst_port', 'total_fwd_pkts', 'total_bwd_pkts', 'flow_bytes_s', 'flow_pkts_s']:
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
    X_val[col] = pd.to_numeric(X_val[col], errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col], errors='coerce')



In [ ]:
for col in ['flow_duration', 'dst_port', 'total_fwd_pkts', 'total_bwd_pkts', 'flow_bytes_s', 'flow_pkts_s']:
    median_val = X_train[col].median()
    X_train[col].fillna(median_val, inplace=True)
    X_val[col].fillna(median_val, inplace=True)
    X_test[col].fillna(median_val, inplace=True)


In [ ]:
import numpy as np

print("Infinite values in X_train:", np.isinf(X_train).sum().sum())
print("Very large values (>1e10):", (X_train > 1e10).sum().sum())


In [ ]:
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_val = X_val.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)


In [ ]:
X_train = X_train.fillna(X_train.median())
X_val   = X_val.fillna(X_train.median())  # use train's median to avoid leakage
X_test  = X_test.fillna(X_train.median())


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)



In [ ]:
# Convert back to DataFrames before saving
X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_val_df   = pd.DataFrame(X_val_scaled, columns=X_val.columns)
X_test_df  = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Add encoded labels
X_train_df['label'] = y_train.values
X_val_df['label']   = y_val.values
X_test_df['label']  = y_test.values


In [ ]:
X_train_df.to_csv("train_scaled.csv", index=False)
X_val_df.to_csv("val_scaled.csv", index=False)
X_test_df.to_csv("test_scaled.csv", index=False)

print("✅ Saved: train_scaled.csv, val_scaled.csv, test_scaled.csv")


In [ ]:
from sklearn.model_selection import train_test_split

# First split: 70% train, 30% temp
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, stratify=y, random_state=42)

# Second split: 15% val, 15% test from 30%
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)


In [ ]:
import joblib

joblib.dump(scaler, "standard_scaler.pkl")
joblib.dump(le_target, "label_encoder.pkl")


In [ ]:
print("Train:\n", y_train.value_counts(normalize=True))
print("Val:\n", y_val.value_counts(normalize=True))
print("Test:\n", y_test.value_counts(normalize=True))


In [ ]:
non_numeric_cols = X_train.select_dtypes(include='object').columns.tolist()
print("Non-numeric columns:", non_numeric_cols)


In [ ]:
# List of known numeric columns with invalid string values
cols_to_fix = ['flow_duration', 'dst_port', 'total_fwd_pkts', 'total_bwd_pkts', 'flow_bytes_s', 'flow_pkts_s']

# Convert to numeric (invalid values → NaN)
for col in cols_to_fix:
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
    X_val[col] = pd.to_numeric(X_val[col], errors='coerce')
    X_test[col] = pd.to_numeric(X_test[col], errors='coerce')

# Replace NaN with median (from train set only)
for col in cols_to_fix:
    median_val = X_train[col].median()
    X_train[col].fillna(median_val, inplace=True)
    X_val[col].fillna(median_val, inplace=True)
    X_test[col].fillna(median_val, inplace=True)


In [ ]:
print("NaNs per column in X_train:")
print(X_train.isna().sum())


In [ ]:
for col in ['src_bytes', 'dst_bytes']:
    median_val = X_train[col].median()
    X_train[col].fillna(median_val, inplace=True)
    X_val[col].fillna(median_val, inplace=True)
    X_test[col].fillna(median_val, inplace=True)


In [ ]:
print("Remaining NaNs:", X_train.isna().sum().sum())


In [ ]:
import numpy as np

print("Infs in X_train:", np.isinf(X_train).sum().sum())
print("Values > 1e10 in X_train:", (X_train > 1e10).sum().sum())


In [ ]:
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_val = X_val.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)


In [ ]:
X_train = X_train.fillna(X_train.median())
X_val = X_val.fillna(X_train.median())  # use train's median for consistency
X_test = X_test.fillna(X_train.median())


In [ ]:
print("✅ NaNs remaining:", X_train.isna().sum().sum())
print("✅ Infs remaining:", np.isinf(X_train).sum().sum())


In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_sm_scaled = scaler.fit_transform(X_train_sm)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


In [ ]:
import joblib
joblib.dump(scaler, "standard_scaler.pkl")


In [ ]:
# Save compressed CSVs instead
X_train_df.to_csv("train_dl.csv.gz", index=False, compression="gzip")
X_val_df.to_csv("val_dl.csv.gz", index=False, compression="gzip")
X_test_df.to_csv("test_dl.csv.gz", index=False, compression="gzip")

print("✅ Saved compressed CSVs: train_dl.csv.gz, val_dl.csv.gz, test_dl.csv.gz")



 Apply SMOTE with Timer & Parallel Jobs

In [ ]:
from imblearn.over_sampling import SMOTE
import time

# Remove n_jobs parameter
smote = SMOTE(random_state=42)
start = time.time()
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)
end = time.time()
print(f"✅ SMOTE completed in {end - start:.2f} seconds.")

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_sm_scaled = scaler.fit_transform(X_train_sm)
X_val_scaled      = scaler.transform(X_val)
X_test_scaled     = scaler.transform(X_test)


In [ ]:
import joblib
joblib.dump(scaler, "standard_scaler.pkl")


In [ ]:
import pandas as pd

X_train_df = pd.DataFrame(X_train_sm_scaled, columns=X_train.columns)
X_val_df   = pd.DataFrame(X_val_scaled, columns=X_val.columns)
X_test_df  = pd.DataFrame(X_test_scaled, columns=X_test.columns)

X_train_df['label'] = y_train_sm
X_val_df['label']   = y_val.values
X_test_df['label']  = y_test.values


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp train_dl.csv.gz /content/drive/MyDrive/
!cp val_dl.csv.gz /content/drive/MyDrive/
!cp test_dl.csv.gz /content/drive/MyDrive/


Plot Class Distribution

In [ ]:
import numpy as np
print("Unique labels after SMOTE:", np.unique(y_train_sm))
print("Total samples:", len(y_train_sm))


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(df['attack_type'])


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# ✅ Sample just 100,000 rows for plotting
y_sample = pd.Series(y_train_sm).sample(100_000, random_state=42)

# ✅ Map integer labels to attack names using your LabelEncoder
label_to_attack = dict(enumerate(le.classes_))
attack_names = y_sample.map(label_to_attack)

# ✅ Wrap into DataFrame for plotting
y_sample_df = pd.DataFrame({'attack_type': attack_names})

# ✅ Plot
plt.figure(figsize=(20, 8))
sns.countplot(data=y_sample_df, x='attack_type', order=y_sample_df['attack_type'].value_counts().index, palette='tab20')
plt.title("Sampled Class Distribution After SMOTE (100,000 rows)", fontsize=16)
plt.xlabel("Attack Type")
plt.ylabel("Count")
plt.xticks(rotation=90, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 📌 Mapping label indices back to original attack type names
label_to_name = dict(enumerate(le.classes_))

# 🔹 Map encoded labels to attack type names
y_train_named = y_train.map(label_to_name)
y_sm_named_sample = pd.Series(y_train_sm).sample(100_000, random_state=42).map(label_to_name)

# 🔹 Create DataFrames for plotting
y_train_df = pd.DataFrame({'Attack Type': y_train_named})
y_sm_df = pd.DataFrame({'Attack Type': y_sm_named_sample})

# 🔹 Set plotting style
sns.set(style="whitegrid")

# 📊 Create side-by-side subplots
fig, axes = plt.subplots(1, 2, figsize=(24, 10), sharey=True)

# ▶️ Before SMOTE
sns.countplot(data=y_train_df, y='Attack Type',
              order=y_train_df['Attack Type'].value_counts().index,
              ax=axes[0], palette="viridis")
axes[0].set_title("Before SMOTE", fontsize=14)
axes[0].set_xlabel("Sample Count", fontsize=12)
axes[0].set_ylabel("Attack Type", fontsize=12)

# ▶️ After SMOTE (sampled)
sns.countplot(data=y_sm_df, y='Attack Type',
              order=y_sm_df['Attack Type'].value_counts().index,
              ax=axes[1], palette="viridis")
axes[1].set_title("After SMOTE (Sample of 100,000)", fontsize=14)
axes[1].set_xlabel("Sample Count", fontsize=12)
axes[1].set_ylabel("")  # Remove y-axis label to avoid repetition

# 🔠 Overall plot title
plt.suptitle("Comparison of Attack Type Distribution — Before and After SMOTE Oversampling", fontsize=18)
plt.tight_layout(rect=[0, 0, 1, 0.96])

# 💾 Optional: Save as PNG for thesis figures
plt.savefig("attack_distribution_before_after_smote.png", dpi=300)

# 📊 Show plot
plt.show()


Time Series or Flow Duration **Distribution**



In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(df['flow_duration'].sample(100_000, random_state=42), bins=100, kde=True)
plt.title("Flow Duration Distribution (Sample of 100K)")
plt.xlabel("Flow Duration")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()



 Feature Correlation Heatmap

In [ ]:
plt.figure(figsize=(14, 12))
corr = df.select_dtypes(include='number').corr()
sns.heatmap(corr, cmap='coolwarm', annot=False, fmt=".2f")
plt.title("Feature Correlation Heatmap")
plt.show()


t-SNE or PCA 2D Visualization

In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# Scale
# 1. Create a copy of X to avoid modifying the original DataFrame
X_temp = X[:5000].copy()

# 2. If 'protocol' is in the DataFrame, encode it to numerical
if 'protocol' in X_temp.columns:
    le = LabelEncoder()
    X_temp['protocol'] = le.fit_transform(X_temp['protocol'])

# 3. Convert hexadecimal strings to integers before scaling
X_temp = X_temp.apply(pd.to_numeric, errors='coerce').fillna(0) # Convert hex to int and fill NaN with 0

# 4. Now apply StandardScaler to the modified DataFrame
X_scaled = StandardScaler().fit_transform(X_temp)
y_sample = y[:5000]

# Reduce
X_embedded = TSNE(n_components=2, perplexity=30, random_state=42).fit_transform(X_scaled)

# Plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_embedded[:, 0], y=X_embedded[:, 1], hue=pd.Series(y_sample).map(label_to_name), palette='tab10', s=50, alpha=0.8)
plt.title("t-SNE Projection of Sampled Features")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

Missing Data Heatmap

In [ ]:
import missingno as msno
msno.matrix(df.sample(10000))


Feature Value Distributions (Histogram Grid)

In [ ]:
important_features = ['flow_duration', 'src_bytes', 'dst_bytes', 'flow_bytes_s', 'total_fwd_pkts', 'total_bwd_pkts']

df[important_features].hist(bins=50, figsize=(12, 8))
plt.suptitle("Feature Distributions Before Scaling", fontsize=16)
plt.tight_layout()
plt.show()


Class Imbalance Before SMOTE (Pie Chart)

 t-SNE / PCA Clustering Visualization

In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import pandas as pd  # Import pandas if not already imported

# 1. Create a copy of X to avoid modifying the original DataFrame
X_temp = X[:5000].copy()

# 2. If 'protocol' is in the DataFrame, encode it to numerical
if 'protocol' in X_temp.columns:
    le = LabelEncoder()
    X_temp['protocol'] = le.fit_transform(X_temp['protocol'])

# 3. Convert hexadecimal strings in 'dst_port' to integers
X_temp['dst_port'] = X_temp['dst_port'].apply(lambda x: int(x, 16) if isinstance(x, str) and x.startswith('0x') else x)

# 4. Convert all columns to numeric (invalid values → NaN)
X_temp = X_temp.apply(pd.to_numeric, errors='coerce')

# 5. Fill NaN with median (from this subset only)
X_temp = X_temp.fillna(X_temp.median())

# 6. Now apply StandardScaler to the modified DataFrame
X_scaled = StandardScaler().fit_transform(X_temp)
y_sample = y[:5000]

# ***DIAGNOSE***: Print number of NaNs to check
print("Number of NaNs in X_scaled:", np.isnan(X_scaled).sum())

# ***FIX***: Impute NaNs using SimpleImputer if any are found
if np.isnan(X_scaled).sum() > 0:
    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(strategy='median')  # Or other strategies like 'mean', 'most_frequent'
    X_scaled = imputer.fit_transform(X_scaled)


# ... (rest of your t-SNE code)
X_embedded = TSNE(n_components=2, random_state=42, perplexity=30).fit_transform(X_scaled)

plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_embedded[:,0], y=X_embedded[:,1], hue=pd.Series(y[:5000]).map(label_to_name), palette='tab10', s=40)
plt.title("t-SNE Visualization of Sampled Attack Types")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

Correlation Heatmap (Features)


In [ ]:
plt.figure(figsize=(14, 10))
sns.heatmap(pd.DataFrame(X_train_sm, columns=X.columns).corr(), cmap="coolwarm", linewidths=0.5)
plt.title("Feature Correlation Heatmap (Train Set)")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.heatmap(
    df[top_features].corr(),
    annot=True,
    fmt=".2f",
    cmap='coolwarm',
    square=True,
    linewidths=0.5,
    annot_kws={"size": 10, "color": "black"}
)

plt.title("Feature Correlation Heatmap", fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig("feature_correlation_heatmap_annotated.png", dpi=300)
plt.show()


In [ ]:
important_features = ['flow_duration', 'src_bytes', 'dst_bytes', 'flow_bytes_s', 'total_fwd_pkts', 'total_bwd_pkts']
df[important_features].sample(100_000).hist(bins=50, figsize=(12, 8), edgecolor='black')
plt.suptitle("Distributions of Important Features (Sample of 100K)", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
df[top_features].isna().sum()


In [ ]:
# Step 1: List your selected features
top_features = [
    'flow_duration', 'src_bytes', 'dst_bytes',
    'total_fwd_pkts', 'total_bwd_pkts',
    'flow_bytes_s', 'flow_pkts_s'
]

# Step 2: Force all values to numeric (convert strings to floats, NaNs stay)
df_imputed = df[top_features].apply(pd.to_numeric, errors='coerce')

# Step 3: Median imputation
df_imputed = df_imputed.fillna(df_imputed.median())

# Step 4: Sample 50,000 for fast plotting
df_sample = df_imputed.sample(50000, random_state=42)

# Step 5: Plot
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20, 12))
for i, feature in enumerate(top_features):
    plt.subplot(3, 3, i + 1)
    sns.histplot(df_sample[feature], bins=50, kde=True, color='darkcyan')
    plt.title(f"{feature} Distribution", fontsize=12)
    plt.xlabel(feature)
    plt.ylabel("Frequency")

plt.suptitle("📊 Feature Distributions After Type Fix + Median Imputation (Sampled 50K)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig("feature_distribution_fixed.png", dpi=300)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler

# ─── 1. Define top features ─────────────────────────────
top_features = [
    'flow_duration', 'src_bytes', 'dst_bytes',
    'total_fwd_pkts', 'total_bwd_pkts',
    'flow_bytes_s', 'flow_pkts_s'
]

# ─── 2. Convert to numeric ───────────────────────────────
df_numeric = df[top_features].apply(pd.to_numeric, errors='coerce')

# ─── 3. Replace inf and extreme values ──────────────────
df_numeric = df_numeric.replace([np.inf, -np.inf], np.nan)

# ─── 4. Fill NaNs with median ───────────────────────────
df_imputed = df_numeric.fillna(df_numeric.median())

# ─── 5. Sample 100,000 for visualization ────────────────
df_sample = df_imputed.sample(100_000, random_state=42)

# ─── 6. Scale data ──────────────────────────────────────
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_sample), columns=top_features)

# ─── 7. Plot Before & After Scaling ─────────────────────
fig, axes = plt.subplots(len(top_features), 2, figsize=(14, 4 * len(top_features)))
sns.set(style="whitegrid")

for i, feature in enumerate(top_features):
    sns.histplot(df_sample[feature], bins=50, kde=True, ax=axes[i, 0], color='cornflowerblue')
    axes[i, 0].set_title(f"{feature} — Before Scaling")
    axes[i, 0].set_xlabel(feature)

    sns.histplot(df_scaled[feature], bins=50, kde=True, ax=axes[i, 1], color='seagreen')
    axes[i, 1].set_title(f"{feature} — After Scaling (StandardScaler)")
    axes[i, 1].set_xlabel(feature)

plt.suptitle("📊 Feature Distributions — Before vs After Standard Scaling", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.savefig("feature_distributions_scaling_comparison.png", dpi=300)
plt.show()


Model Training: CNN + BiLSTM + Attention

In [ ]:
import pandas as pd

# Load training set
train_df = pd.read_csv("/content/train_dl.csv.gz")
val_df = pd.read_csv("/content/val_dl.csv.gz")
test_df = pd.read_csv("/content/test_dl.csv.gz")

# Count top 10 frequent labels
top_N = 10
top_labels = train_df['label'].value_counts().nlargest(top_N).index.tolist()

# Filter only top N classes
train_df = train_df[train_df['label'].isin(top_labels)]
val_df   = val_df[val_df['label'].isin(top_labels)]
test_df  = test_df[test_df['label'].isin(top_labels)]

# Re-map labels to 0..(N-1)
label_map = {old: new for new, old in enumerate(sorted(top_labels))}
train_df['label'] = train_df['label'].map(label_map)
val_df['label']   = val_df['label'].map(label_map)
test_df['label']  = test_df['label'].map(label_map)

# Save filtered files
train_df.to_csv("train_topN.csv", index=False)
val_df.to_csv("val_topN.csv", index=False)
test_df.to_csv("test_topN.csv", index=False)

print("✅ Saved: train_topN.csv, val_topN.csv, test_topN.csv")


In [ ]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

# Load
train_df = pd.read_csv("/content/train_topN.csv")
val_df   = pd.read_csv("/content/val_topN.csv")
test_df  = pd.read_csv("/content/test_topN.csv")

# Inputs and labels
X_train = train_df.drop(columns=["label"]).values
y_train = train_df["label"].values
X_val   = val_df.drop(columns=["label"]).values
y_val   = val_df["label"].values
X_test  = test_df.drop(columns=["label"]).values
y_test  = test_df["label"].values

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor   = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor   = torch.tensor(y_val, dtype=torch.long)
X_test_tensor  = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test, dtype=torch.long)

# Dataloaders
batch_size = 512
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size)
test_loader  = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size)

# Number of classes (should match top_N)
num_classes = len(np.unique(y_train))


In [ ]:
import torch.nn as nn

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        weights = torch.softmax(self.attn(x), dim=1)
        context = torch.sum(weights * x, dim=1)
        return context

class CNN_BiLSTM_Attention(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 128, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.bilstm = nn.LSTM(128, hidden_dim, bidirectional=True, batch_first=True)
        self.attn = Attention(hidden_dim * 2)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)            # (B, 1, F)
        x = self.relu(self.conv1(x)) # (B, 128, F)
        x = x.permute(0, 2, 1)       # (B, T, D)
        x, _ = self.bilstm(x)
        context = self.attn(x)
        return self.fc(context)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Class weights for imbalanced classes
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

model = CNN_BiLSTM_Attention(input_dim=X_train.shape[1], hidden_dim=256, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)


In [ ]:
best_val_loss = float('inf')
patience = 10
wait = 0

for epoch in range(1, 50):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)

    train_acc = correct / total
    val_loss, val_correct, val_total = 0, 0, 0
    model.eval()
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            val_loss += criterion(out, yb).item()
            val_preds = torch.argmax(out, dim=1)
            val_correct += (val_preds == yb).sum().item()
            val_total += yb.size(0)

    val_acc = val_correct / val_total
    val_loss /= len(val_loader)

    print(f"Epoch {epoch:02d} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Val Loss: {val_loss:.4f}")

    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pt")
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print("⛔ Early stopping triggered.")
            break


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix

# Step 1: Compute confusion matrix
cm = confusion_matrix(all_labels, all_preds)
classes = ['dos', 'ddos', 'injection', 'mitm', 'normal', 'password', 'scanning', 'xss', 'backdoor', 'ransomware']

# Step 2: Normalize (for % values)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Step 3: Plot with seaborn heatmap
plt.figure(figsize=(12, 10))
sns.set(font_scale=1.2)  # adjust font size
ax = sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues", xticklabels=classes, yticklabels=classes, linewidths=0.5, square=True)

# Step 4: Labels and formatting
plt.ylabel('True Label', fontsize=14)
plt.xlabel('Predicted Label', fontsize=14)
plt.title('Normalized Confusion Matrix', fontsize=16)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()

# Step 5: Save for thesis use (e.g., PDF, PNG)
plt.savefig("confusion_matrix_thesis.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
from sklearn.metrics import classification_report
import pandas as pd
import seaborn as sns

report = classification_report(all_labels, all_preds, target_names=classes, output_dict=True)
df_report = pd.DataFrame(report).transpose()
sns.heatmap(df_report.iloc[:-1, :-1], annot=True, cmap="YlGnBu")
plt.title("Precision, Recall, F1-score per Class")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import RocCurveDisplay
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import label_binarize

# Get predictions for the test set
model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    model_outputs = []
    all_labels = []
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        outputs = model(xb)
        model_outputs.append(outputs)  # Store model outputs
        all_labels.append(yb)
                          # Concatenate predictions and labels

# FIX: Concatenate all_labels and model_outputs outside the loop
model_outputs = torch.cat(model_outputs, dim=0)
all_labels = torch.cat(all_labels, dim=0)

y_test_bin = label_binarize(all_labels.cpu(), classes=range(len(classes)))
y_score = model_outputs.detach().cpu().numpy()  # use softmax/logits output from model

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(classes)):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
plt.figure(figsize=(10, 8))
for i in range(len(classes)):
    plt.plot(fpr[i], tpr[i], label=f"{classes[i]} (AUC = {roc_auc[i]:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Multi-Class ROC Curve")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:
import torch
import matplotlib.pyplot as plt

# Initialize early stopping and tracking variables
best_val_loss = float('inf')
patience = 10
wait = 0

# Lists for plotting
train_acc_list, val_acc_list = [], []
train_loss_list, val_loss_list = [], []

for epoch in range(1, 50):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)

    train_acc = correct / total
    train_loss = total_loss / len(train_loader)
    train_acc_list.append(train_acc)
    train_loss_list.append(train_loss)

    # Validation
    model.eval()
    val_loss_total, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            outputs = model(xb)
            loss = criterion(outputs, yb)
            val_loss_total += loss.item() * yb.size(0)  # weight by batch size
            preds = torch.argmax(outputs, dim=1)
            val_correct += (preds == yb).sum().item()
            val_total += yb.size(0)

    val_acc = val_correct / val_total
    val_loss = val_loss_total / val_total
    val_acc_list.append(val_acc)
    val_loss_list.append(val_loss)

    # Logging
    print(f"Epoch {epoch:02d} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Val Loss: {val_loss:.4f}")

    # Step learning rate scheduler
    scheduler.step(val_loss)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pt")
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print("⛔ Early stopping triggered.")
            break


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import torch
import matplotlib.pyplot as plt
import numpy as np

# Load best model
model.load_state_dict(torch.load("best_model.pt"))
model.eval()

# Evaluate on test set
all_preds = []
all_labels = []

with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        outputs = model(xb)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(yb.cpu().numpy())

# Compute Test Accuracy
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
test_acc = np.mean(all_preds == all_labels)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

# Classification Report
print("\n📋 Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes))

# Confusion Matrix (Normalized)
cm = confusion_matrix(all_labels, all_preds)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plot Confusion Matrix
plt.figure(figsize=(10, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm_norm, display_labels=classes)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45, colorbar=True)
plt.title("Normalized Confusion Matrix - Test Set")
plt.tight_layout()
plt.grid(False)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np

# ─── Replace with your actual class names ───────────────────────────
class_names = ['dos', 'ddos', 'injection', 'mitm', 'normal',
               'password', 'scanning', 'xss', 'backdoor', 'ransomware']

# Assuming 'all_labels' are your true labels and 'all_preds' are your predictions
# ─── Generate confusion matrix ──────────────────────────────────────
cm = confusion_matrix(all_labels, all_preds, labels=range(len(class_names)))

# ─── Normalize if needed ────────────────────────────────────────────
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# ─── Plot ───────────────────────────────────────────────────────────
plt.figure(figsize=(12, 10))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Proportion'}, linewidths=0.5, square=True)

plt.title("Normalized Confusion Matrix", fontsize=16)
plt.xlabel("Predicted Label", fontsize=14)
plt.ylabel("True Label", fontsize=14)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig("confusion_matrix_thesis.png", dpi=300)
plt.show()

In [ ]:
# Accuracy Plot
plt.figure(figsize=(10, 4))
plt.plot(train_acc_list, label='Train Accuracy', marker='o')
plt.plot(val_acc_list, label='Validation Accuracy', marker='s')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Train vs Validation Accuracy")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Loss Plot
plt.figure(figsize=(10, 4))
plt.plot(train_loss_list, label='Train Loss', marker='o')
plt.plot(val_loss_list, label='Validation Loss', marker='s')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Train vs Validation Loss")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Get number of classes
n_classes = len(classes)

# Convert labels to one-hot
y_true_bin = label_binarize(all_labels, classes=range(n_classes))

# Get predicted probabilities
model.eval()
y_scores = []
y_true_raw = []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        outputs = model(xb)
        probs = torch.softmax(outputs, dim=1)  # probabilities
        y_scores.append(probs.cpu().numpy())
        y_true_raw.append(yb.cpu().numpy())

y_scores = np.vstack(y_scores)
y_true_bin = label_binarize(np.concatenate(y_true_raw), classes=range(n_classes))

# Compute ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_scores[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot
plt.figure(figsize=(10, 8))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f"{classes[i]} (AUC = {roc_auc[i]:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.title("Multi-Class ROC Curves (One-vs-Rest)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.grid(True)
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Sample a subset
np.random.seed(42)
sample_indices = np.random.choice(len(labels), size=5000, replace=False)
sampled_features = features[sample_indices]
sampled_labels = labels[sample_indices]

# Apply t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
features_2d = tsne.fit_transform(sampled_features)

# Plot
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=features_2d[:, 0],
    y=features_2d[:, 1],
    hue=sampled_labels,
    palette='tab10',
    s=60,
    alpha=0.8,
    legend='full'
)
plt.title("t-SNE of Test Set Feature Embeddings (Sampled)", fontsize=15)
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.legend(title="Class", loc="best", fontsize=9)
plt.grid(True)
plt.tight_layout()
plt.savefig("tsne_testset.png", dpi=300)
plt.show()



In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Sample a subset if needed (reuse from t-SNE step)
np.random.seed(42)
sample_indices = np.random.choice(len(labels), size=5000, replace=False)
sampled_features = features[sample_indices]
sampled_labels = labels[sample_indices]

# Apply PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(sampled_features)

# Plot
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=pca_result[:, 0],
    y=pca_result[:, 1],
    hue=sampled_labels,
    palette='tab10',
    s=60,
    alpha=0.8,
    legend='full'
)
plt.title("PCA of Test Set Feature Embeddings (Sampled)", fontsize=15)
plt.xlabel(f"PC 1 ({pca.explained_variance_ratio_[0]*100:.2f}%)")
plt.ylabel(f"PC 2 ({pca.explained_variance_ratio_[1]*100:.2f}%)")
plt.legend(title="Class", loc="best", fontsize=9)
plt.grid(True)
plt.tight_layout()
plt.savefig("pca_testset.png", dpi=300)
plt.show()


BiLSTM Model Setup (Full Feature Set)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
class BiLSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers=1, dropout=0.3):
        super(BiLSTMClassifier, self).__init__()
        self.bilstm = nn.LSTM(input_size=input_size,
                              hidden_size=hidden_size,
                              num_layers=num_layers,
                              batch_first=True,
                              bidirectional=True,
                              dropout=dropout if num_layers > 1 else 0.0)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add seq_len=1
        out, _ = self.bilstm(x)
        out = out[:, -1, :]
        return self.fc(out)


In [ ]:
# Replace with your actual datasets
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=256, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=256, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=256, shuffle=False)


In [ ]:
input_size = X_train.shape[1]
model = BiLSTMClassifier(input_size=input_size, hidden_size=128, num_classes=10).to(device)

# Class weights
weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(weights, dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

best_val_loss = float('inf')
wait = 0
patience = 5

for epoch in range(1, 31):
    model.train()
    total, correct, total_loss = 0, 0, 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (out.argmax(1) == yb).sum().item()
        total += yb.size(0)

    train_acc = correct / total

    # Validation
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            val_loss += criterion(out, yb).item()
            val_correct += (out.argmax(1) == yb).sum().item()
            val_total += yb.size(0)

    val_acc = val_correct / val_total
    val_loss /= len(val_loader)
    scheduler.step(val_loss)

    print(f"Epoch {epoch:02d} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Val Loss: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        wait = 0
        torch.save(model.state_dict(), "best_bilstm.pt")
    else:
        wait += 1
        if wait >= patience:
            print("⏹️ Early stopping.")
            break


In [ ]:
# Load best model
model.load_state_dict(torch.load("best_bilstm.pt"))
model.eval()

y_true, y_pred, y_prob = [], [], []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        out = model(xb)
        probs = torch.softmax(out, dim=1)
        preds = torch.argmax(probs, dim=1)

        y_true.extend(yb.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_prob.extend(probs.cpu().numpy())

# Classification report
print("✅ Test Accuracy:", np.mean(np.array(y_true) == np.array(y_pred)))
print("\n📋 Classification Report:\n", classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

cm = confusion_matrix(y_true, y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(12, 10))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Proportion'}, linewidths=0.5)
plt.title("BiLSTM Confusion Matrix (Normalized)", fontsize=15)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("bilstm_confusion_matrix.png", dpi=300)
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt

# Binarize labels for One-vs-Rest AUC
y_true_bin = label_binarize(y_true, classes=list(range(len(class_names))))
y_prob = np.array(y_prob)

plt.figure(figsize=(12, 10))
for i in range(len(class_names)):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_prob[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{class_names[i]} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.title("BiLSTM ROC Curves — Multi-class (One-vs-Rest)", fontsize=15)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right", fontsize=10)
plt.grid(True)
plt.tight_layout()
plt.savefig("bilstm_roc_curves.png", dpi=300)
plt.show()


In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns

# Sample 5000 points for t-SNE
np.random.seed(42)
idx = np.random.choice(len(y_true), 5000, replace=False)
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
tsne_results = tsne.fit_transform(y_prob[idx])

plt.figure(figsize=(10, 8))
sns.scatterplot(x=tsne_results[:, 0], y=tsne_results[:, 1],
                hue=np.array(y_true)[idx], palette='tab10', s=50, alpha=0.8, legend='full')
plt.title("t-SNE of BiLSTM Test Set Embeddings")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.legend(title="Class", loc="best", fontsize=9)
plt.grid(True)
plt.tight_layout()
plt.savefig("bilstm_tsne.png", dpi=300)
plt.show()


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_results = pca.fit_transform(y_prob[idx])

plt.figure(figsize=(10, 8))
sns.scatterplot(x=pca_results[:, 0], y=pca_results[:, 1],
                hue=np.array(y_true)[idx], palette='tab10', s=50, alpha=0.8, legend='full')
plt.title("PCA of BiLSTM Test Set Embeddings")
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.2f}%)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.2f}%)")
plt.legend(title="Class", loc="best", fontsize=9)
plt.grid(True)
plt.tight_layout()
plt.savefig("bilstm_pca.png", dpi=300)
plt.show()


Full LSTM Model Pipeline (Using Normal Dataset)

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers=1, dropout=0.3):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True,
                            dropout=dropout if num_layers > 1 else 0.0)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # Shape: (batch, seq_len=1, features)
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # Use last time step
        return self.fc(out)


In [ ]:
input_size = X_train.shape[1]  # Full feature count
model = LSTMClassifier(input_size=input_size, hidden_size=128, num_classes=10).to(device)

from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)


In [ ]:
best_val_loss = float('inf')
wait = 0
patience = 5

for epoch in range(1, 31):
    model.train()
    total, correct, total_loss = 0, 0, 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (out.argmax(1) == yb).sum().item()
        total += yb.size(0)

    train_acc = correct / total

    # Validation
    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            val_loss += criterion(out, yb).item()
            val_correct += (out.argmax(1) == yb).sum().item()

    val_acc = val_correct / len(val_loader.dataset)
    val_loss /= len(val_loader)
    scheduler.step(val_loss)

    print(f"Epoch {epoch:02d} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Val Loss: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_lstm.pt")
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print("⏹️ Early stopping.")
            break


In [ ]:
model.load_state_dict(torch.load("best_lstm.pt"))


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA


In [ ]:
# Load best LSTM model
model.load_state_dict(torch.load("best_lstm.pt"))
model.eval()

y_true, y_pred, y_prob = [], [], []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        out = model(xb)
        probs = torch.softmax(out, dim=1)
        preds = torch.argmax(probs, dim=1)

        y_true.extend(yb.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_prob.extend(probs.cpu().numpy())

# Convert to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_prob = np.array(y_prob)


In [ ]:
print(f"✅ Test Accuracy: {accuracy_score(y_true, y_pred):.4f}")
print("\n📋 Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
cm = confusion_matrix(y_true, y_pred)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(12, 10))
sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Proportion'})
plt.title("LSTM Confusion Matrix (Normalized)", fontsize=15)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("lstm_confusion_matrix.png", dpi=300)
plt.show()


In [ ]:
y_true_bin = label_binarize(y_true, classes=range(len(class_names)))

plt.figure(figsize=(12, 10))
for i in range(len(class_names)):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_prob[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{class_names[i]} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.title("LSTM ROC Curves — Multi-class (One-vs-Rest)", fontsize=15)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right", fontsize=10)
plt.grid(True)
plt.tight_layout()
plt.savefig("lstm_roc_curves.png", dpi=300)
plt.show()


In [ ]:
# Sample 5000 points for visualization
np.random.seed(42)
sample_idx = np.random.choice(len(y_true), size=5000, replace=False)

# t-SNE
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
tsne_result = tsne.fit_transform(y_prob[sample_idx])

plt.figure(figsize=(10, 8))
sns.scatterplot(x=tsne_result[:, 0], y=tsne_result[:, 1],
                hue=y_true[sample_idx], palette='tab10', s=50, alpha=0.8)
plt.title("t-SNE of LSTM Test Set Embeddings")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.legend(title="Class", loc="best", fontsize=9)
plt.tight_layout()
plt.savefig("lstm_tsne.png", dpi=300)
plt.show()

# PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(y_prob[sample_idx])

plt.figure(figsize=(10, 8))
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1],
                hue=y_true[sample_idx], palette='tab10', s=50, alpha=0.8)
plt.title("PCA of LSTM Test Set Embeddings")
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.2f}%)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.2f}%)")
plt.legend(title="Class", loc="best", fontsize=9)
plt.tight_layout()
plt.savefig("lstm_pca.png", dpi=300)
plt.show()


CNN + BiLSTM Model

In [ ]:
import torch.nn as nn

class CNN_BiLSTM(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNN_BiLSTM, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)

        self.bilstm = nn.LSTM(input_size=64, hidden_size=128,
                              num_layers=1, batch_first=True,
                              bidirectional=True)

        self.fc = nn.Linear(128 * 2, num_classes)

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch, 1, features)
        x = self.conv1(x)   # (batch, 64, features)
        x = self.relu(x)
        x = self.pool(x)    # (batch, 64, features/2)
        x = x.permute(0, 2, 1)  # (batch, seq_len, features) for LSTM
        out, _ = self.bilstm(x)
        out = out[:, -1, :]  # last time step
        return self.fc(out)


In [ ]:
input_size = X_train.shape[1]
model = CNN_BiLSTM(input_size=input_size, num_classes=10).to(device)

from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)


In [ ]:
best_val_loss = float('inf')
wait = 0
patience = 5

for epoch in range(1, 31):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (out.argmax(1) == yb).sum().item()
        total += yb.size(0)

    train_acc = correct / total

    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            val_loss += criterion(out, yb).item()
            val_correct += (out.argmax(1) == yb).sum().item()

    val_acc = val_correct / len(val_loader.dataset)
    val_loss /= len(val_loader)
    scheduler.step(val_loss)

    print(f"Epoch {epoch:02d} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Val Loss: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        wait = 0
        torch.save(model.state_dict(), "best_cnn_bilstm.pt")
    else:
        wait += 1
        if wait >= patience:
            print("⏹️ Early stopping.")
            break


In [ ]:
model.load_state_dict(torch.load("best_cnn_bilstm.pt"))
model.eval()



In [ ]:
model.load_state_dict(torch.load("best_cnn_bilstm.pt"))
model.eval()

y_true, y_pred, y_prob = [], [], []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        out = model(xb)
        probs = torch.softmax(out, dim=1)
        preds = torch.argmax(probs, dim=1)

        y_true.extend(yb.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_prob.extend(probs.cpu().numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_prob = np.array(y_prob)


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

print(f"✅ Test Accuracy: {accuracy_score(y_true, y_pred):.4f}")
print("\n📋 Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(y_true, y_pred)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(12, 10))
sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Proportion'})
plt.title("CNN + BiLSTM Confusion Matrix (Normalized)", fontsize=15)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("cnn_bilstm_confusion_matrix.png", dpi=300)
plt.show()


In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

y_true_bin = label_binarize(y_true, classes=range(len(class_names)))

plt.figure(figsize=(12, 10))
for i in range(len(class_names)):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_prob[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{class_names[i]} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.title("CNN + BiLSTM ROC Curves — Multi-class", fontsize=15)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right", fontsize=10)
plt.grid(True)
plt.tight_layout()
plt.savefig("cnn_bilstm_roc_curves.png", dpi=300)
plt.show()


In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import numpy as np

# Sample 5000 points for visualization
np.random.seed(42)
sample_idx = np.random.choice(len(y_true), size=5000, replace=False)

# t-SNE
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
tsne_result = tsne.fit_transform(y_prob[sample_idx])

plt.figure(figsize=(10, 8))
sns.scatterplot(x=tsne_result[:, 0], y=tsne_result[:, 1],
                hue=y_true[sample_idx], palette='tab10', s=50, alpha=0.8)
plt.title("t-SNE of CNN + BiLSTM Test Embeddings")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.legend(title="Class", loc="best", fontsize=9)
plt.tight_layout()
plt.savefig("cnn_bilstm_tsne.png", dpi=300)
plt.show()

# PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(y_prob[sample_idx])

plt.figure(figsize=(10, 8))
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1],
                hue=y_true[sample_idx], palette='tab10', s=50, alpha=0.8)
plt.title("PCA of CNN + BiLSTM Test Embeddings")
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.2f}%)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.2f}%)")
plt.legend(title="Class", loc="best", fontsize=9)
plt.tight_layout()
plt.savefig("cnn_bilstm_pca.png", dpi=300)
plt.show()


 MLP Model — Full Code (With Class Weights & Evaluation)

In [ ]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)


In [ ]:
input_size = X_train.shape[1]
model = MLP(input_size=input_size, num_classes=10).to(device)

from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)


In [ ]:
best_val_loss = float('inf')
wait = 0
patience = 5

for epoch in range(1, 31):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (out.argmax(1) == yb).sum().item()
        total += yb.size(0)

    train_acc = correct / total

    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            val_loss += criterion(out, yb).item()
            val_correct += (out.argmax(1) == yb).sum().item()

    val_acc = val_correct / len(val_loader.dataset)
    val_loss /= len(val_loader)
    scheduler.step(val_loss)

    print(f"Epoch {epoch:02d} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f} | Val Loss: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_mlp.pt")
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print("⏹️ Early stopping.")
            break


In [ ]:
model.load_state_dict(torch.load("best_mlp.pt"))
model.eval()

y_true, y_pred, y_prob = [], [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        out = model(xb)
        probs = torch.softmax(out, dim=1)
        preds = torch.argmax(probs, dim=1)
        y_true.extend(yb.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_prob.extend(probs.cpu().numpy())



In [ ]:
from sklearn.metrics import classification_report, accuracy_score

print(f"✅ Test Accuracy: {accuracy_score(y_true, y_pred):.4f}")
print("\n📋 Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(12, 10))
sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Proportion'})
plt.title("MLP Confusion Matrix (Normalized)", fontsize=15)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("mlp_confusion_matrix.png", dpi=300)
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np

y_true_bin = label_binarize(y_true, classes=range(len(class_names)))
y_prob = np.array(y_prob)

plt.figure(figsize=(12, 10))
for i in range(len(class_names)):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_prob[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{class_names[i]} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.title("MLP ROC Curves — Multi-class", fontsize=15)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right", fontsize=10)
plt.grid(True)
plt.tight_layout()
plt.savefig("mlp_roc_curves.png", dpi=300)
plt.show()


In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# Sample 5000 for visual clarity
np.random.seed(42)
sample_idx = np.random.choice(len(y_true), size=5000, replace=False)

# t-SNE
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
tsne_result = tsne.fit_transform(y_prob[sample_idx])

plt.figure(figsize=(10, 8))
# FIX: Convert y_true to a NumPy array for proper indexing
sns.scatterplot(x=tsne_result[:, 0], y=tsne_result[:, 1],
                hue=np.array(y_true)[sample_idx], palette='tab10', s=50, alpha=0.8)
plt.title("t-SNE of MLP Test Embeddings")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.legend(title="Class", loc="best", fontsize=9)
plt.tight_layout()
plt.savefig("mlp_tsne.png", dpi=300)
plt.show()

# PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(y_prob[sample_idx])

plt.figure(figsize=(10, 8))
# FIX: Convert y_true to a NumPy array for proper indexing
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1],
                hue=np.array(y_true)[sample_idx], palette='tab10', s=50, alpha=0.8)
plt.title("PCA of MLP Test Embeddings")
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.2f}%)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.2f}%)")
plt.legend(title="Class", loc="best", fontsize=9)
plt.tight_layout()
plt.savefig("mlp_pca.png", dpi=300)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Model names
models = ['MLP', 'LSTM', 'BiLSTM', 'CNN+BiLSTM', 'CNN+BiLSTM+Attn']

# Metrics
accuracy = [0.61, 0.698, 0.702, 0.869, 0.887]
macro_f1 = [0.55, 0.64, 0.64, 0.79, 0.82]

x = np.arange(len(models))  # label locations
width = 0.35  # width of bars


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, accuracy, width, label='Accuracy')
bars2 = ax.bar(x + width/2, macro_f1, width, label='Macro F1')

# Add value labels
for bar in bars1 + bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.2f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),  # vertical offset
                textcoords="offset points",
                ha='center', va='bottom')

# Axis formatting
ax.set_ylabel('Score')
ax.set_title('DL Model Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(models, rotation=15)
ax.set_ylim(0, 1.05)
ax.legend()
ax.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()

# Save plot
plt.savefig("dl_model_comparison.png", dpi=300)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# Actual class names used in dataset
class_names = [
    'dos', 'ddos', 'injection', 'mitm', 'normal',
    'password', 'scanning', 'xss', 'backdoor', 'ransomware'
]

# Simulated data placeholders (replace with real output from your models)
models = ['MLP', 'LSTM', 'BiLSTM', 'CNN+BiLSTM', 'CNN+BiLSTM+Attn']
accuracy = [0.61, 0.698, 0.702, 0.869, 0.887]
macro_f1 = [0.55, 0.64, 0.64, 0.79, 0.82]
conf_matrices = [np.random.rand(10, 10) for _ in models]
roc_data = [np.linspace(0, 1, 100) for _ in models]
tsne_data = [np.random.randn(500, 2) for _ in models]
pca_data = [np.random.randn(500, 2) for _ in models]
labels = np.random.randint(0, 10, 500)

# ─────────────────────────────────────────────────────────────────────────────
# Bar Plot: Accuracy vs Macro F1
x = np.arange(len(models))
width = 0.35
fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, accuracy, width, label='Accuracy')
bars2 = ax.bar(x + width/2, macro_f1, width, label='Macro F1')
for bar in bars1 + bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.2f}', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
ax.set_ylabel('Score')
ax.set_title('DL Model Comparison: Accuracy vs Macro F1')
ax.set_xticks(x)
ax.set_xticklabels(models, rotation=15)
ax.set_ylim(0, 1.05)
ax.legend()
ax.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# ─────────────────────────────────────────────────────────────────────────────
# Confusion Matrices (first 3 models)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for i, ax in enumerate(axes):
    sns.heatmap(conf_matrices[i], ax=ax, cmap='Blues', annot=False,
                xticklabels=class_names, yticklabels=class_names)
    ax.set_title(f"{models[i]} Confusion Matrix")
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
plt.tight_layout()
plt.show()

# ─────────────────────────────────────────────────────────────────────────────
# ROC Curves
plt.figure(figsize=(10, 8))
for i, model in enumerate(models):
    fpr = roc_data[i]
    tpr = roc_data[i] ** (1 + 0.1*i)  # simulate different shapes
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{model} (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.title("ROC Curve Comparison Across Models")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()

# ─────────────────────────────────────────────────────────────────────────────
# t-SNE Embeddings
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for i, ax in enumerate(axes):
    sns.scatterplot(x=tsne_data[i][:, 0], y=tsne_data[i][:, 1],
                    hue=[class_names[l] for l in labels],
                    palette='tab10', ax=ax, legend=False)
    ax.set_title(f"t-SNE: {models[i]}")
    ax.set_xlabel("Dim 1")
    ax.set_ylabel("Dim 2")
plt.tight_layout()
plt.show()

# ─────────────────────────────────────────────────────────────────────────────
# PCA Embeddings
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for i, ax in enumerate(axes):
    sns.scatterplot(x=pca_data[i][:, 0], y=pca_data[i][:, 1],
                    hue=[class_names[l] for l in labels],
                    palette='tab10', ax=ax, legend=False)
    ax.set_title(f"PCA: {models[i]}")
    ax.set_xlabel("PC1")
    ax.set_ylabel("PC2")
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Radar chart setup
labels = ['Accuracy', 'Precision', 'Recall', 'Macro F1']
num_vars = len(labels)

# Sample model metrics (replace with actual if needed)
mlp_metrics =     [0.61, 0.60, 0.65, 0.55]
lstm_metrics =    [0.698, 0.69, 0.72, 0.64]
bilstm_metrics =  [0.702, 0.70, 0.73, 0.64]
cnn_bilstm =      [0.869, 0.87, 0.88, 0.79]
cnn_bilstm_attn = [0.887, 0.89, 0.90, 0.82]

model_scores = [mlp_metrics, lstm_metrics, bilstm_metrics, cnn_bilstm, cnn_bilstm_attn]
model_names = ['MLP', 'LSTM', 'BiLSTM', 'CNN+BiLSTM', 'CNN+BiLSTM+Attn']

# Radar chart angles
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]

# Create plot
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

# Plot each model
for scores, name in zip(model_scores, model_names):
    data = scores + scores[:1]
    ax.plot(angles, data, label=name)
    ax.fill(angles, data, alpha=0.1)

# Labels and legend
ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)
ax.set_thetagrids(np.degrees(angles[:-1]), labels)
ax.set_ylim(0, 1.1)
plt.title("Radar Chart: DL Model Performance", size=16)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
plt.tight_layout()
plt.show()


ml models

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Train model
rf_model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced', n_jobs=-1)
rf_model.fit(X_train, y_train)

# Predict on test set
y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)

# Basic results
print(f"✅ Test Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred_rf, target_names=class_names))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

cm = confusion_matrix(y_test, y_pred_rf, normalize='true')

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title("Random Forest — Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

# Binarize true labels
y_test_bin = label_binarize(y_test, classes=np.arange(len(class_names)))

plt.figure(figsize=(12, 8))
for i in range(len(class_names)):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_prob_rf[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{class_names[i]} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.title("Random Forest — ROC Curve (Multiclass)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right", fontsize=9)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Assuming you have a DataFrame called 'df' that you used to create X_train
feature_names = df.drop(columns=['attack_type', 'label_enc']).columns  # Get feature names from original DataFrame

# Train model
rf_model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced', n_jobs=-1)
rf_model.fit(X_train, y_train)

# Predict on test set
y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)

# Basic results
print(f"✅ Test Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred_rf, target_names=class_names))

# ... (rest of your code)

importances = rf_model.feature_importances_
indices = np.argsort(importances)[-15:]  # top 15

plt.figure(figsize=(10, 6))
plt.barh(range(len(indices)), importances[indices], align='center')

# Use feature_names instead of X_train.columns
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])

plt.xlabel("Feature Importance")
plt.title("Random Forest — Top 15 Important Features")
plt.tight_layout()
plt.show()

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

# Train model
xgb_model = XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=10,
                          objective='multi:softprob', num_class=10,
                          eval_metric='mlogloss', use_label_encoder=False,
                          n_jobs=-1, random_state=42)
xgb_model.fit(X_train, y_train)

# Predict on test set
y_pred_xgb = xgb_model.predict(X_test)
y_prob_xgb = xgb_model.predict_proba(X_test)

# Evaluation
print(f"✅ Test Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred_xgb, target_names=class_names))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

cm = confusion_matrix(y_test, y_pred_xgb, normalize='true')

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title("XGBoost — Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

y_test_bin = label_binarize(y_test, classes=np.arange(len(class_names)))

plt.figure(figsize=(12, 8))
for i in range(len(class_names)):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_prob_xgb[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{class_names[i]} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.title("XGBoost — ROC Curve (Multiclass)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right", fontsize=9)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import xgboost as xgb  # Import the xgboost module

plt.figure(figsize=(10, 6))
# Call plot_importance from xgboost, passing the booster
xgb.plot_importance(xgb_model.get_booster(), importance_type='gain', max_num_features=15)
plt.title("XGBoost — Top 15 Important Features")
plt.tight_layout()
plt.show()


In [ ]:
import lightgbm as lgb
from sklearn.metrics import classification_report, accuracy_score

# Train model
lgb_model = lgb.LGBMClassifier(objective='multiclass',
                               num_class=10,
                               n_estimators=200,
                               learning_rate=0.1,
                               max_depth=10,
                               class_weight='balanced',
                               random_state=42,
                               n_jobs=-1)

lgb_model.fit(X_train, y_train)

# Predict
y_pred_lgb = lgb_model.predict(X_test)
y_prob_lgb = lgb_model.predict_proba(X_test)

# Evaluation
print(f"✅ Test Accuracy: {accuracy_score(y_test, y_pred_lgb):.4f}")
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred_lgb, target_names=class_names))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

cm = confusion_matrix(y_test, y_pred_lgb, normalize='true')

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='.2f', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title("LightGBM — Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

y_test_bin = label_binarize(y_test, classes=np.arange(len(class_names)))

plt.figure(figsize=(12, 8))
for i in range(len(class_names)):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_prob_lgb[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{class_names[i]} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.title("LightGBM — ROC Curve (Multiclass)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right", fontsize=9)
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
lgb.plot_importance(lgb_model, max_num_features=15, importance_type='gain')
plt.title("LightGBM — Top 15 Important Features")
plt.tight_layout()
plt.show()


plots for thesis

In [ ]:
# Install gdown silently
!pip install --quiet gdown

# Import libraries
import gdown
import pandas as pd

# Updated Google Drive shareable link
url = "https://drive.google.com/uc?id=1t0mqjuEHOu_WZWh6wI_As1KvdZWHixHv"
output = "new_dataset.csv"  # You can rename this as needed

# Download the file
gdown.download(url, output, quiet=False)

# Load the CSV into a DataFrame
df = pd.read_csv(output)
print("✅ Shape of the dataset:", df.shape)
df.head()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title("Missing Values Heatmap")
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load uploaded dataset
df = pd.read_csv("/content/new_dataset.csv")

# Step 1: Clean attack_type labels
df['attack_type'] = df['attack_type'].str.lower().replace({
    'udp-lag': 'udplag', 'udp_lag': 'udplag',
    'drdos_mssql': 'drdos_mssql', 'drdos_msssql': 'drdos_mssql',
    'drdos_dns': 'drdos_dns', 'drdos_snmp': 'drdos_snmp',
    'drdos_ldap': 'drdos_ldap', 'drdos_netbios': 'drdos_netbios',
    'normal': 'benign', 'webddos': 'webddos'
})

# Count attacks
attack_counts = df['attack_type'].value_counts()

# Plot 1: Log-scaled full attack distribution
plt.figure(figsize=(18, 8))
bars = plt.bar(attack_counts.index, attack_counts.values, color='steelblue')
plt.yscale('log')

for bar in bars:
    height = bar.get_height()
    if height > 0:
        plt.text(bar.get_x() + bar.get_width()/2, height * 1.1,
                 f'{int(height):,}', ha='center', va='bottom', fontsize=9, rotation=90)

plt.title('Log-Scaled Distribution of Cleaned Attack Types', fontsize=18, fontweight='bold')
plt.xlabel('Attack Type', fontsize=14)
plt.ylabel('Number of Samples (Log Scale)', fontsize=14)
plt.xticks(rotation=75, ha='right', fontsize=11)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()


In [ ]:
import pandas as pd

# Load raw combined dataset
df_raw = pd.read_csv("/content/new_dataset.csv")  # Replace with your actual path
print("✅ Raw dataset shape:", df_raw.shape)
df_raw.head()


In [ ]:
print("🔍 Raw Dataset Info:")
print(df_raw.info())
print("\n🔍 Missing values (before cleaning):")
print(df_raw.isnull().sum())
print("\n🔍 Class Distribution (before):")
print(df_raw['attack_type'].value_counts())


In [ ]:
print("📋 Current Columns:", df.columns.tolist())


In [ ]:
# Step 1: Reload raw dataset
df_raw = pd.read_csv("/content/new_dataset.csv", low_memory=False)
df_raw.columns = df_raw.columns.str.strip().str.lower()

# Step 2: Clean attack_type column
df = df_raw.copy()
df['attack_type'] = df['attack_type'].astype(str).str.strip().str.upper()
df['attack_type'] = df['attack_type'].replace({'NAN': np.nan, '1.0': 'BENIGN', '0.0': 'BENIGN', 'LABEL': np.nan})

# Step 3: Drop only rows without attack_type
df = df.dropna(subset=['attack_type'])

# Step 4: Convert object → numeric
numeric_columns = ['flow_duration', 'dst_port', 'total_fwd_pkts', 'total_bwd_pkts', 'flow_bytes_s', 'flow_pkts_s']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Step 5: Impute missing numeric values with median
for col in numeric_columns:
    median_val = df[col].median()
    df[col].fillna(median_val, inplace=True)

# Step 6: Encode protocol (categorical)
df['protocol'], _ = pd.factorize(df['protocol'])

# Step 7: Encode attack_type
from sklearn.preprocessing import LabelEncoder
le_target = LabelEncoder()
df['attack_type_enc'] = le_target.fit_transform(df['attack_type'])

# ✅ Diagnostic: Print shape
print("✅ Cleaned dataset shape:", df.shape)

# Step 7.5: Replace inf/-inf with NaN, then impute with median
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(df.median(numeric_only=True), inplace=True)


# Step 8: Plot class distribution before SMOTE
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.countplot(y='attack_type', data=df, order=df['attack_type'].value_counts().index)
plt.title("📊 Class Distribution Before Preprocessing")
plt.tight_layout()
plt.show()

# Step 9: Feature scaling
from sklearn.preprocessing import StandardScaler
features = df.drop(columns=['attack_type', 'attack_type_enc'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

# Step 10: Apply SMOTE
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_scaled, df['attack_type_enc'])

# Step 11: After SMOTE plot
df_after = pd.DataFrame(X_resampled, columns=features.columns)
df_after['attack_type_enc'] = y_resampled

plt.figure(figsize=(12, 6))
sns.countplot(x='attack_type_enc', data=df_after, order=sorted(df_after['attack_type_enc'].unique()))
plt.title("✅ Class Distribution After SMOTE + Encoding")
plt.tight_layout()
plt.show()

# Step 12: Label Mapping
label_mapping = dict(zip(le_target.transform(le_target.classes_), le_target.classes_))
print("📌 Encoded Label Mapping:")
for k, v in label_mapping.items():
    print(f"{k}: {v}")
